# Preparation

In [ ]:
# Loading data from Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
#PyTorch lib
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torch.nn.functional as F
import torchvision
#Tools lib
import numpy as np
import cv2
import random
import time
import os
import argparse
from matplotlib import pyplot as plt
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import sys

root_path = '/content/drive/MyDrive/DeRaindrop-master/'
model_path = '/content/drive/MyDrive/DeRaindrop-master/models'
weights_path = '/content/drive/MyDrive/DeRaindrop-master/weights/gen.pkl' # trained by Qian et al.
#weights_path = "/content/drive/MyDrive/Qian/weights/finetuned_model_0815.pt" # my model

sys.path.append(root_path)
sys.path.append(model_path)

from models import *
#from metrics import *
from predict import align_to_four

def predict(image):
    image = np.array(image, dtype='float32')/255.
    image = image.transpose((2, 0, 1))
    image = image[np.newaxis, :, :, :]
    image = torch.from_numpy(image)
    image = Variable(image).cuda()

    out = model(image)[-1]

    out = out.cpu().data
    out = out.numpy()
    out = out.transpose((0, 2, 3, 1))
    out = out[0, :, :, :]*255.

    return out


def run(mode, input_dir, output_dir='', gt_dir='', output_flag=False):

    if mode == 'demo':
        input_list = sorted(os.listdir(input_dir))
        num = len(input_list)
        for i in range(num):
            print ('Processing image: %s'%(input_list[i]))
            img = cv2.imread(input_dir + input_list[i])

            #img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            #plt.imshow(img_rgb)
            #plt.show()

            img = align_to_four(img)
            result = predict(img)
            #print(img.shape, result.shape)
            img_name = input_list[i].split('.')[0]
            cv2.imwrite(output_dir + img_name + '.jpg', result)
            #result_rgb = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
            #plt.imshow(result_rgb)
            #plt.show()

    elif mode == 'test':
        input_list = os.listdir(input_dir)
        num = len(input_list)
        cumulative_psnr = 0
        cumulative_ssim = 0

        for i in range(num):
            input_file_name = input_list[i]
            if 'rain' in input_list[i]:
                gt_file_name = input_file_name.replace('rain', 'clean')
            elif 'right' in input_list[i]:
                gt_file_name = input_file_name.replace('right', 'left')
            else:
                print("ERROR")

            input = cv2.imread(input_dir + input_file_name)
            gt = cv2.imread(gt_dir + gt_file_name)
            input = align_to_four(input)
            gt = align_to_four(gt)
            result = predict(input)
            if output_flag:
                img_name = input_list[i].split('.')[0]
                cv2.imwrite(output_dir + img_name + '.jpg', result)
            result = np.array(result, dtype = 'uint8')
            cur_psnr = psnr(gt, result)
            im1_y = cv2.cvtColor(gt, cv2.COLOR_BGR2YCR_CB)[:, :, 0]
            im2_y = cv2.cvtColor(result, cv2.COLOR_BGR2YCR_CB)[:, :, 0]
            cur_ssim = ssim(im1_y, im2_y)
            cumulative_psnr += cur_psnr
            cumulative_ssim += cur_ssim
        print('In testing dataset, PSNR is %.4f and SSIM is %.4f'%(cumulative_psnr/num, cumulative_ssim/num))

    elif mode == 'compare_without_predict':
        # To see how much a image is blurred
        input_list = os.listdir(input_dir)
        num = len(input_list)
        cumulative_psnr = 0
        cumulative_ssim = 0

        for i in range(num):
            input_file_name = input_list[i]
            if 'rain' in input_list[i]:
                gt_file_name = input_file_name.replace('rain', 'clean')
            elif 'right' in input_list[i]:
                gt_file_name = input_file_name.replace('right', 'left')
            else:
                print("ERROR")

            input = cv2.imread(input_dir + input_file_name)
            gt = cv2.imread(gt_dir + gt_file_name)
            input = align_to_four(input)
            gt = align_to_four(gt)
            cur_psnr = psnr(gt, input)
            im1_y = cv2.cvtColor(gt, cv2.COLOR_BGR2YCR_CB)[:, :, 0]
            im2_y = cv2.cvtColor(input, cv2.COLOR_BGR2YCR_CB)[:, :, 0]
            cur_ssim = ssim(im1_y, im2_y)
            cumulative_psnr += cur_psnr
            cumulative_ssim += cur_ssim
        print('In testing dataset, PSNR is %.4f and SSIM is %.4f'%(cumulative_psnr/num, cumulative_ssim/num))

    else:
        print ('Mode Invalid!')

# att+M

In [ ]:
test_input_dir = '/content/drive/MyDrive/dataset/test/data/'
test_gt_dir = '/content/drive/MyDrive/dataset/test/gt/'
test_output_dir = '/content/drive/MyDrive/Qian/att+M/output/'

weights_path = "/content/drive/MyDrive/Qian/att+M/weights/59.pt"
model = Generator().cuda()
model.load_state_dict(torch.load(weights_path))
run(mode='test', input_dir=test_input_dir,
                    output_dir=test_output_dir,
                    gt_dir=test_gt_dir,
                    output_flag = True)

In testing dataset, PSNR is 13.7260 and SSIM is 0.6381


In [ ]:
test_input_dir = '/content/drive/MyDrive/dataset/test/data/'
test_gt_dir = '/content/drive/MyDrive/dataset/test/gt/'
test_output_dir = '/content/drive/MyDrive/Qian/att+M/output/'

weights_path = "/content/drive/MyDrive/Qian/att+M/weights/30.pt"
model = Generator().cuda()
model.load_state_dict(torch.load(weights_path))
run(mode='test', input_dir=test_input_dir,
                    output_dir=test_output_dir,
                    gt_dir=test_gt_dir,
                    output_flag = True)

In testing dataset, PSNR is 14.4464 and SSIM is 0.6455


# att + P

In [ ]:
test_input_dir = '/content/drive/MyDrive/dataset/test/data/'
test_gt_dir = '/content/drive/MyDrive/dataset/test/gt/'
test_output_dir = '/content/drive/MyDrive/Qian/att+P/output/'

weights_path = "/content/drive/MyDrive/Qian/att+P/weights/59.pt"
model = Generator().cuda()
model.load_state_dict(torch.load(weights_path))
run(mode='test', input_dir=test_input_dir,
                    output_dir=test_output_dir,
                    gt_dir=test_gt_dir,
                    output_flag = True)

In testing dataset, PSNR is 16.7576 and SSIM is 0.6862


# all Loss

In [ ]:
test_input_dir = '/content/drive/MyDrive/dataset/test/data/'
test_gt_dir = '/content/drive/MyDrive/dataset/test/gt/'
test_output_dir = '/content/drive/MyDrive/Qian/all_loss/output/'

weights_path = "/content/drive/MyDrive/Qian/all_loss/all_loss.pt"
model = Generator().cuda()
model.load_state_dict(torch.load(weights_path))
run(mode='test', input_dir=test_input_dir,
                    output_dir=test_output_dir,
                    gt_dir=test_gt_dir,
                    output_flag = False)

In testing dataset, PSNR is 16.1567 and SSIM is 0.6808


## All loss 1

In [ ]:
test_input_dir = '/content/drive/MyDrive/dataset/test/data/'
test_gt_dir = '/content/drive/MyDrive/dataset/test/gt/'
test_output_dir = '/content/drive/MyDrive/Qian/all_loss_1/output/'

weights_path = "/content/drive/MyDrive/Qian/all_loss_1/weights/59.pt"
model = Generator().cuda()
model.load_state_dict(torch.load(weights_path))
run(mode='test', input_dir=test_input_dir,
                    output_dir=test_output_dir,
                    gt_dir=test_gt_dir,
                    output_flag = True)

In testing dataset, PSNR is 16.4368 and SSIM is 0.6847


# L_P

In [ ]:
test_input_dir = '/content/drive/MyDrive/dataset/test/data/'
test_gt_dir = '/content/drive/MyDrive/dataset/test/gt/'
test_output_dir = '/content/drive/MyDrive/Qian/L_P/output/'

weights_path = "/content/drive/MyDrive/Qian/L_P/L_P.pt"
model = Generator().cuda()
model.load_state_dict(torch.load(weights_path))
run(mode='test', input_dir=test_input_dir,
                    output_dir=test_output_dir,
                    gt_dir=test_gt_dir,
                    output_flag = False)

In testing dataset, PSNR is 16.7275 and SSIM is 0.6857


# L_att

In [ ]:
test_input_dir = '/content/drive/MyDrive/dataset/combined/test/data/'
test_gt_dir = '/content/drive/MyDrive/dataset/combined/test/gt/'
test_output_dir = '/content/drive/MyDrive/Qian/L_att/output/'

weights_path = "/content/drive/MyDrive/Qian/L_att/L_att.pt"
model = Generator().cuda()
model.load_state_dict(torch.load(weights_path))
run(mode='test', input_dir=test_input_dir,
                    output_dir=test_output_dir,
                    gt_dir=test_gt_dir,
                    output_flag = True)

In testing dataset, PSNR is 16.1914 and SSIM is 0.6614


## L_att_1

In [ ]:
test_input_dir = '/content/drive/MyDrive/dataset/test/data/'
test_gt_dir = '/content/drive/MyDrive/dataset/test/gt/'
test_output_dir = '/content/drive/MyDrive/Qian/L_att_1/output/'

weights_path = "/content/drive/MyDrive/Qian/L_att_1/weights/70.pt"
model = Generator().cuda()
model.load_state_dict(torch.load(weights_path))
run(mode='test', input_dir=test_input_dir,
                    output_dir=test_output_dir,
                    gt_dir=test_gt_dir,
                    output_flag = True)

In testing dataset, PSNR is 14.7172 and SSIM is 0.6283


# L_M

In [ ]:
test_input_dir = '/content/drive/MyDrive/dataset/test/data/'
test_gt_dir = '/content/drive/MyDrive/dataset/test/gt/'
test_output_dir = '/content/drive/MyDrive/Qian/L_M/output/'

weights_path = "/content/drive/MyDrive/Qian/L_M/L_M.pt"
model = Generator().cuda()
model.load_state_dict(torch.load(weights_path))
run(mode='test', input_dir=test_input_dir,
                    output_dir=test_output_dir,
                    gt_dir=test_gt_dir,
                    output_flag = False)

In testing dataset, PSNR is 13.2484 and SSIM is 0.6286


## L_M_1

In [ ]:
test_input_dir = '/content/drive/MyDrive/dataset/test/data/'
test_gt_dir = '/content/drive/MyDrive/dataset/test/gt/'
test_output_dir = '/content/drive/MyDrive/Qian/L_M/output/'

weights_path = "/content/drive/MyDrive/Qian/L_M_1/weights/66.pt"
model = Generator().cuda()
model.load_state_dict(torch.load(weights_path))
run(mode='test', input_dir=test_input_dir,
                    output_dir=test_output_dir,
                    gt_dir=test_gt_dir,
                    output_flag = True)

In testing dataset, PSNR is 13.3972 and SSIM is 0.6314


In [ ]:
test_input_dir = '/content/drive/MyDrive/dataset/test/data/'
test_gt_dir = '/content/drive/MyDrive/dataset/test/gt/'
test_output_dir = '/content/drive/MyDrive/Qian/L_M/output/'

weights_path = "/content/drive/MyDrive/Qian/L_M_1/weights/30.pt"
model = Generator().cuda()
model.load_state_dict(torch.load(weights_path))
run(mode='test', input_dir=test_input_dir,
                    output_dir=test_output_dir,
                    gt_dir=test_gt_dir,
                    output_flag = False)

In testing dataset, PSNR is 13.9677 and SSIM is 0.6372


# Model B trained on big droplets (Multiple loss)

In [ ]:
test_input_dir = '/content/drive/MyDrive/dataset/combined/test/data/'
test_gt_dir = '/content/drive/MyDrive/dataset/combined/test/gt/'
test_output_dir = '/content/drive/MyDrive/dataset/combined/test/output_v9/'

weights_path = "/content/drive/MyDrive/Qian/weights/weights_v9.pt"
model = Generator().cuda()
model.load_state_dict(torch.load(weights_path))
run(mode='test', input_dir=test_input_dir,
                    output_dir=test_output_dir,
                    gt_dir=test_gt_dir,
                    output_flag = False)

In [ ]:
test_input_dir = '/content/drive/MyDrive/dataset/test/data/'
test_gt_dir = '/content/drive/MyDrive/dataset/test/gt/'
test_output_dir = '/content/drive/MyDrive/dataset/test/output_model_A/'
weights_path = "/content/drive/MyDrive/Qian/weights/weights_v9.pt"
model = Generator().cuda()
model.load_state_dict(torch.load(weights_path))
run(mode='test', input_dir=test_input_dir,
                    output_dir=test_output_dir,
                    gt_dir=test_gt_dir,
                    output_flag = True)

In testing dataset, PSNR is 16.1588 and SSIM is 0.6816


# model A (trained with single loss function)

In [ ]:
test_input_dir = '/content/drive/MyDrive/dataset/combined/test/data/'
test_gt_dir = '/content/drive/MyDrive/dataset/combined/test/gt/'
test_output_dir = '/content/drive/MyDrive/dataset/combined/test/output_trained/'

weights_path = "/content/drive/MyDrive/Qian/weights/finetuned_model_0815.pt"
model = Generator().cuda()
model.load_state_dict(torch.load(weights_path))
run(mode='test', input_dir=test_input_dir,
                    output_dir=test_output_dir,
                    gt_dir=test_gt_dir,
                    output_flag = False)

In testing dataset, PSNR is 15.2320 and SSIM is 0.6675


In [ ]:
test_input_dir = '/content/drive/MyDrive/dataset/test/data/'
test_gt_dir = '/content/drive/MyDrive/dataset/test/gt/'
test_output_dir = '/content/drive/MyDrive/dataset/test/output_model_A/'
weights_path = "/content/drive/MyDrive/Qian/weights/finetuned_model_0815.pt"
model = Generator().cuda()
model.load_state_dict(torch.load(weights_path))
run(mode='test', input_dir=test_input_dir,
                    output_dir=test_output_dir,
                    gt_dir=test_gt_dir,
                    output_flag = True)

In testing dataset, PSNR is 13.6848 and SSIM is 0.6375


# Model 0 (Qian's model)

In [ ]:
test_input_dir = '/content/drive/MyDrive/dataset/combined/test/data/'
test_gt_dir = '/content/drive/MyDrive/dataset/combined/test/gt/'
test_output_dir = '/content/drive/MyDrive/dataset/combined/test/output_origin/'

weights_path = '/content/drive/MyDrive/DeRaindrop-master/weights/gen.pkl'
model = Generator().cuda()
model.load_state_dict(torch.load(weights_path))
run(mode='test', input_dir=test_input_dir,
                    output_dir=test_output_dir,
                    gt_dir=test_gt_dir,
                    output_flag = False)

In testing dataset, PSNR is 16.2754 and SSIM is 0.6630


In [ ]:
test_input_dir = '/content/drive/MyDrive/dataset/test/data/'
test_gt_dir = '/content/drive/MyDrive/dataset/test/gt/'
test_output_dir = '/content/drive/MyDrive/dataset/test/output_model_A/'
weights_path = '/content/drive/MyDrive/DeRaindrop-master/weights/gen.pkl'
model = Generator().cuda()
model.load_state_dict(torch.load(weights_path))
run(mode='test', input_dir=test_input_dir,
                    output_dir=test_output_dir,
                    gt_dir=test_gt_dir,
                    output_flag = True)

In testing dataset, PSNR is 14.6256 and SSIM is 0.6266


# To compare two datasets
to see qualitatively how the images are blured in both datasets

In [ ]:
# small raindrop (Fine)
test_input_dir = '/content/drive/MyDrive/dataset/Qian/test_b/data/'
test_gt_dir = '/content/drive/MyDrive/dataset/Qian/test_b/gt/'
print("Small")
run(mode='compare_without_predict', input_dir=test_input_dir,gt_dir=test_gt_dir)

# large raindrop
test_input_dir = '/content/drive/MyDrive/dataset/test/data/'
test_gt_dir = '/content/drive/MyDrive/dataset/test/gt/'
print("Large")
run(mode='compare_without_predict', input_dir=test_input_dir,gt_dir=test_gt_dir)

Small
In testing dataset, PSNR is 21.4183 and SSIM is 0.7616
Large
In testing dataset, PSNR is 12.6757 and SSIM is 0.5450


# Demo


In [ ]:
test_input_dir = '/content/drive/MyDrive/DeRaindrop-master/demo/input/'
test_output_dir = '/content/drive/MyDrive/DeRaindrop-master/demo/output/'
weights_path = "/content/drive/MyDrive/Qian/L_P/L_P.pt"
model = Generator().cuda()
model.load_state_dict(torch.load(weights_path))
run(mode='demo', input_dir=test_input_dir,
                    output_dir=test_output_dir,
                    output_flag = False)

Processing image: 451_rain.png
Processing image: 962_rain.png
